# Linear SVM Classification

SVM은 다음의 그림을 통해 잘 설명될 수 있다.

<br/>
<img src="./images/large_margin.png" alt="large_margin"> 
<br/>

이 그림은 앞절에서 살펴본 iris datset을 보여준다. 그림에서 보이는 것처럼, 직선을 통해 완전히 분류될 수 있다.(**linearly separable**)

왼편의 그림을 보면, 3개의 linear classifier의 decision boundary들이 있다. dash line의 경우 분류를 제대로 하지 못하는 잘못된 경우이며, 나머지 두 line은 분류는 잘 하지만 instance들과 매우 가까워서 새로운 instance에 대해서는 잘 동작하지 않을 수 있다.

반면, 오른편의 그림에서는 decision boundary가 class를 잘 분류하고 있으며, instance와 가장 멀리 떨어진 위치에 있다는 것을 알 수 있다. 이 decision boundary가 SVM classifier이며 이는 class간의 widest possible street에 fitting하는 방식이라고 생각할 수 있다. 이를 **large margin classification**이라고 한다.

SVM을 자세히 살펴보면, off the street(길 바깥)에 training instance를 더 추가하더라도 decision boundary에 영향을 주지 않으며 edge of the street(길 테두리)에 위치한 instance들에 의해서만 determined(or supported) 된다는 것을 알 수 있다. 이러한 instance들을 **support vector**라고 부른다.(오른편 그림에서 동그라미 쳐진 instance들)

SVM은 feature scale에 민감하다. 아래 그림에서, 왼쪽 그림은 수직축의 scale(x1축, 20간격)이 수평축의 scale(x0축, 1간격)보다 더 커서 decision boundary가 수평에 가깝게 나타났다. 하지만 사이킷런의 `StandardScaler` 등을 사용해 feature scaling을 수행하면, 오른쪽의 그림과 같이, decision boundary가 더욱 좋은 형태가 된다.

<br/>
<img src="./images/sensitivity_to_feature_scales.png" alt="sensitivity_to_feature_scales"> 
<br/>

## Soft Margin Classification

모든 instance들이 street에서 벗어나도록 제한하는 것을 **hard margin classification**이라고 한다. 여기에는 2가지 main issue가 있다. 첫번째는 linearly separable한 경우에만 동작한다는 것이고, 두번째는 outlier에 아주 민감하다는 것이다.

<br/>
<img src="./images/hard_margin_sensitivity_to_outliers.png" alt="hard_margin_sensitivity_to_outliers"> 
<br/>

위 그림의 왼편은 첫번째 issue에 해당한다. linearly separable하지 않으므로 hard margin 방법으로는 decision bounary를 그릴 수 없다. 오른편의 경우는 두번째 issue이며, outlier때문에 decision boundary가 한쪽의 data에 매우 가깝게 그려져서 generalization이 잘 되지 않을 것이다.

위와 같은 issue를 피하기 위한 더 유연한 모델이 있다. 이 모델은 street을 넓게 하는 것과 **margin violation**(instace가 street의 중앙에 있거나, 반대편에 있는 것)간의 balance를 유지한다. 이를 **soft margin classification**이라고 한다.

사이킷런의 SVM class에서는 `C` hyperparameter를 통해 위 balance를 조절할 수 있다. `C`가 작아지면 street은 더 넓어지지만, margin violation도 더 커진다. 아래 그림은 `C`값에 따른 soft margin SVM classifier를 비교한 것이다.

<br/>
<img src="./images/large_margin_versus_fewer_margin.png" alt="large_margin_versus_fewer_margin"> 
<br/>

왼쪽의 경우, `C`값이 작아서 street이 더 넓지만, margin violation도 더 커진 것을 볼 수 있다. 반대로 오른쪽의 경우는 `C`값이 커서 street은 더 좁아졌지만, margin violation이 줄어든 것을 알 수 있다.

위 두 경우를 generalization 관점에서 본다면, 왼쪽의 classifier가 더욱 낫다는 것을 알 수 있다. 따라서, 만약 SVM model이 overfitting된다면, `C`값을 줄이는 것으로 regularization을 시도해 볼 수 있다.

다음의 코드는 iris dataset을 load하고, feature scaling을 수행한 후, linear SVM 모델을 학습시키는 것이다.

In [4]:
import numpy as np
from sklearn import datasets
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC

iris = datasets.load_iris()
X = iris["data"][:, (2, 3)] # petal length / width
y = (iris["target"] == 2).astype(np.float64)

svm_clf = Pipeline([
    ("scaler", StandardScaler()),
    ("linear_svc", LinearSVC(C=1, loss="hinge"))
])

svm_clf.fit(X, y)

# Logistic regression과 달리 SVM classifier는 확률이 아닌 class를 반환
print(svm_clf.predict([[5.5, 1.7]]))

[1.]


위에서 사용한 `LinearSVC` 대신에 `SVC(kernel="linear", C=1)`을 사용할 수 있지만, 더욱 느리므로 추천하지 않는다.

또 다른 방법으로는 `SGDClassifier(loss="hinge", alpha=1/(m*C))`가 있는데, 이는 Stochastic Gradient Descent를 사용해 Linear SVM classifier를 학습시키는 방법이다. 이는 `LinearSVC`보다 converge하는 속도는 느리지만, memory에 올리기 힘든 매우 큰 dataset이나 online learning에 유용하다.

`SVC(kernel="linear")`에 상응하는 `LinearSVC`를 위해서는 먼저, `LinearSVC`가 bias term을 regularize하기 때문에 `StandardScaler`를 사용해 training set을 scaling해주어야 한다. 또한, `loss` hyperparameter의 default가 `hinge`가 아니므로, `loss="hinge"`로 직접 설정해 주어야 한다. 마지막으로, 더 나은 성능을 위해서 (instance의 수보다 feature가 더 많지 않은 경우라면) `dual=False`를 사용할 수 있다.

---

### Hinge loss
함수 $max(0, 1-t)$는 **hinge loss** function이라고 부른다. 이 함수는 $t\ge1$인 경우에 값을 0으로 만들며, 기울기는 $t \lt 1$일때, -1이고, $t \gt 1$일때, 0이다. 다음 그림을 보면 쉽게 이해할 수 있다.

<br/>
<img src="./images/hinge_loss.png" alt="hinge_loss", width="80%"> 
<br/>

Lasso regression에서와 같이 hinge loss에서도 $t=1$에서 미분 불가능지점이 있는데,  subgradient method를 사용하면 Gradient Descent를 사용할 수 있다.

---